In [1]:
import pandas as pd
train_df = pd.read_csv("titanic/train.csv",encoding="utf-8")
test_df = pd.read_csv("titanic/test.csv",encoding="utf-8")

In [2]:
train_df.isna().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [3]:
test_df.isna().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [4]:
med = train_df.median()
train_df = train_df.fillna(med)
test_df = test_df.fillna(med)

In [5]:
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
test_df.isna().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [7]:
most = train_df["Embarked"].value_counts().idxmax()
train_df["Embarked"] = train_df["Embarked"].fillna(most)
test_df["Embarked"] = test_df["Embarked"].fillna(most)

In [8]:
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [9]:
# x_train.isna().sum()

In [10]:
n = "Braund, Mr. Owen Harris"
n.split(",")[-1].split(".")[0]
n.split()

['Braund,', 'Mr.', 'Owen', 'Harris']

In [11]:
def nameflow(n):
    n = n.split(",")[-1].split(".")[0]
    return n.strip()
mid = train_df["Name"].apply(nameflow).value_counts()
reserved = mid[mid > 30].index
reserved

Index(['Mr', 'Miss', 'Mrs', 'Master'], dtype='object')

In [12]:
def nameflow2(n):
    n = n.split(",")[-1].split(".")[0]
    n = n.strip()
    if n in reserved:
        return n
    else:
        return None
    
train_df["Name"] = train_df["Name"].apply(nameflow2)
test_df["Name"] = test_df["Name"].apply(nameflow2)

In [13]:
#dummy_na=True 多name_nan欄位
# pd.get_dummies(train_df, columns=["Name","Sex","Embarked"], dummy_na=True)
x_train_nodrop = pd.get_dummies(train_df,
                              columns=["Name","Sex","Embarked"])
x_test_nodrop = pd.get_dummies(test_df,
                              columns=["Name","Sex","Embarked"])

In [14]:
x_train_nodrop.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,A/5 21171,7.2500,NaN,0,0,1,0,0,1,0,0,1
1,2,1,1,38.0,1,0,PC 17599,71.2833,C85,0,0,0,1,1,0,1,0,0
2,3,1,3,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,0,0,1,0,0,0,1
3,4,1,1,35.0,1,0,113803,53.1000,C123,0,0,0,1,1,0,0,0,1
4,5,0,3,35.0,0,0,373450,8.0500,NaN,0,0,1,0,0,1,0,0,1


In [15]:
x_train = x_train_nodrop.drop(["PassengerId","Survived","Cabin","Ticket"],
                    axis=1)
y_train = x_train_nodrop["Survived"]
x_test = x_test_nodrop.drop(["PassengerId","Cabin","Ticket"],
                    axis=1)
testid = x_test_nodrop["PassengerId"]

In [16]:
x_train

,Pclass,Age,SibSp,Parch,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,0,1,0,0,1,0,0,1
1,1,38.0,1,0,71.2833,0,0,0,1,1,0,1,0,0
2,3,26.0,0,0,7.9250,0,1,0,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,0,0,0,1,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0,0,0,0,0,1,0,0,1
887,1,19.0,0,0,30.0000,0,1,0,0,1,0,0,0,1
888,3,28.0,1,2,23.4500,0,1,0,0,1,0,0,0,1
889,1,26.0,0,0,30.0000,0,0,1,0,0,1,1,0,0


In [17]:
#*args 將tuple內的東西拆開
#**kwargs 將dic內的東西拆開(要照著key順

In [18]:
# def test(a, *args):
#     print(a)
#     print(args)
# test(5,2,1,4,3)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
#看score調整params參數

params = {
    "n_estimators":range(20,110,10),
    "max_depth":range(3,11)
}
clf = RandomForestClassifier()
cv = GridSearchCV(clf, params, cv=10, n_jobs=4)
cv.fit(x_train,y_train)
print(cv.best_params_)
print(cv.best_score_)

{'max_depth': 8, 'n_estimators': 100}
0.8372615039281706


In [20]:
#多跑幾次看數值變化是否穩定
from sklearn.model_selection import cross_val_score
import numpy as np
clf = RandomForestClassifier(n_estimators=26, max_depth=8)
score = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=4)
print(score)
print(np.average(score))


[0.78888889 0.81111111 0.74157303 0.8988764  0.88764045 0.83146067
 0.83146067 0.76404494 0.86516854 0.85227273]
0.8272497446373851


In [21]:
clf = RandomForestClassifier(n_estimators=26, max_depth=8)
clf.fit(x_train,y_train)
pre = clf.predict(x_test)
result_df = pd.DataFrame({
    "PassengerId":testid,
    "survived":pre
})
result_df.to_csv("5_reg.csv", index=False, encoding="utf-8")

In [22]:
#RandomForest importance table
# pd.DataFrame({
#     "column":x_train.columns,
#     "importance":clf.feature_importances_
# })

In [23]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)


In [24]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
params = {
    "n_neighbors":range(5,100)
}
cv = GridSearchCV(clf, params, cv=10, n_jobs=4)
cv.fit(x_train_scale, y_train)
print(cv.best_params_)
print(cv.best_score_)

{'n_neighbors': 17}
0.819304152637486


In [25]:
pre = cv.best_estimator_.predict(x_test_scale)
result_df = pd.DataFrame({
    "PassengerId":testid,
    "survived":pre
})
result_df.to_csv("5_knn.csv", index=False, encoding="utf-8")